In [57]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd

In [58]:
chromedriver = 'C:\\Workspace\\02.DataAnalysis\\05.Crawling&Web\\chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram login

In [59]:
insta_url = 'https://www.instagram.com/'
driver.get(insta_url)
time.sleep(1)

In [60]:
with open('password.txt') as f:
    password = f.read(len('password.txt'))

In [61]:
email = 'codemonkeyyy'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [62]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [63]:
# login click
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [64]:
# alarm click
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. jeju 맛집 search

In [65]:
from urllib.parse import quote
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [66]:
driver.get(url)
time.sleep(5)

### 3. open 1st post

In [67]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)


### 4. get post info

In [68]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [69]:
# 1) get main text info
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content= ' '
content

'협찬제주 대표 썬셋 맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-743-1979영업 : 오전11시~밤10시 연중무휴포장ㆍ배달 환영제주애월맛집💘 랍스터빈#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집 #제주도카페 #제주카페 #외도맛집 #하귀맛집 #한림맛집 #한림카페#애월해안도로맛집 #애월가볼만한곳 #제주공항맛집 #제주공항근처맛집 #제주 #제주도여행'

In [70]:
# 2) get hashtags from main text, '#[^\s#,\\]+'
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#애월맛집',
 '#애월카페',
 '#애월읍맛집',
 '#제주애월맛집',
 '#제주도맛집',
 '#제주맛집',
 '#제주도카페',
 '#제주카페',
 '#외도맛집',
 '#하귀맛집',
 '#한림맛집',
 '#한림카페',
 '#애월해안도로맛집',
 '#애월가볼만한곳',
 '#제주공항맛집',
 '#제주공항근처맛집',
 '#제주',
 '#제주도여행']

In [71]:
# 3) get write day
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-19'

In [72]:
# 4) get count of like
try:
    like = soup.select_one('a.zV_Nj > span').text[:]
except:
    like = 0
like

0

In [73]:
# 5) get location info
try:
    place = soup.select('div.JF9hh')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = 0
place

''

In [74]:
row = [content, date, like, place, tags]
row

['협찬제주 대표 썬셋 맛집 "랍스터빈"주소 : 제주시 애월읍 애월해안로 752전화 : 064-743-1979영업 : 오전11시~밤10시 연중무휴포장ㆍ배달 환영제주애월맛집💘 랍스터빈#애월맛집 #애월카페 #애월읍맛집 #제주애월맛집 #제주도맛집 #제주맛집 #제주도카페 #제주카페 #외도맛집 #하귀맛집 #한림맛집 #한림카페#애월해안도로맛집 #애월가볼만한곳 #제주공항맛집 #제주공항근처맛집 #제주 #제주도여행',
 '2021-08-19',
 0,
 '',
 ['#애월맛집',
  '#애월카페',
  '#애월읍맛집',
  '#제주애월맛집',
  '#제주도맛집',
  '#제주맛집',
  '#제주도카페',
  '#제주카페',
  '#외도맛집',
  '#하귀맛집',
  '#한림맛집',
  '#한림카페',
  '#애월해안도로맛집',
  '#애월가볼만한곳',
  '#제주공항맛집',
  '#제주공항근처맛집',
  '#제주',
  '#제주도여행']]

### 5. move next post

In [75]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(1)

In [76]:
# eliminate pupop window 
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click() 
time.sleep(1)

### 6. get multiple post

In [90]:
from tqdm.notebook import tqdm

In [ ]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content= ' '

    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]

    try:
        like = soup.select_one('a.zV_Nj > span').text[:]
    except:
        like = 0
    
    try:
        place = soup.select('div.JF9hh')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = 0

    return [content, date, like, place, tags]

### - 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [85]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)


In [96]:
results = []
for _ in tqdm(range(500)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

  0%|          | 0/500 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [94]:
df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
df

,content,date,like,place,tags
0,#제주도여행 이건꼭먹어야해💕도시락 포장후 바다뷰보며 먹는 #흑돼지박스자동차안 숙소 ...,2021-08-19,16,,"[#제주도여행, #흑돼지박스자동차안, #제주맛집, #제주도맛집, #산방산맛집, #송..."
1,은쥬투어 Day3⛰🏝#jejuisland,2021-08-19,52,Jeju-do,[#jejuisland]
2,문신돼지국밥충 제주 2일차🏕,2021-08-19,86,황우지해안,[]
3,#food #foodstagram #foodporn #foodie #foodphot...,2021-08-19,18,월정리해변,"[#food, #foodstagram, #foodporn, #foodie, #foo..."
4,Crab Jack in Jeju 🦀🌴🍺-인스타그램에서 보고 꼭 가봐야지 했던 곳! ...,2021-08-19,6,"Jeju Island, South Korea","[#제주도, #제주도맛집, #제주맛집, #제주도여행, #크랩잭, #제주크랩잭, #소..."
...,...,...,...,...,...
96,#하늘 #구름 #바다,2021-08-19,7,,"[#하늘, #구름, #바다]"
97,🎀👚🌷,2021-08-19,54,,[]
98,🖤🤍🖤🤍..#여수 #여수카페 #부산 #부산카페 #제주 #제주도 #제주도맛집 #제주카...,2021-08-19,14,"Seoul, Korea","[#여수, #여수카페, #부산, #부산카페, #제주, #제주도, #제주도맛집, #제..."
99,봉리단길 투어☺️ 전주한옥마을 아님 주의🚫,2021-08-19,46,봉리단길,[]


In [ ]:
df.to_csv('data/제주맛집.csv', index=False)

In [ ]:
for keyword in ['제주도맛집', '제주맛집', '제주도관광', '제주여행']:
    url = f'{search_url}{quote(keyword)}'
    driver.get(url)
    time.sleep(5)
    driver.find_element_by_css_selector('div._9AhH0').click()
    time.sleep(1)

    results = []
    for _ in tqdm(range(200)):
        try:
            row = get_content(driver)
            results.append(row)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
        except:
            time.sleep(3)
            driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
            time.sleep(2)
    df = pd.DataFrame(results, columns=['content', 'date', 'like', 'place', 'tags'])
    df.to_csv(f'data/{keyword}.csv', index=False)

    driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click() 
    time.sleep(2)